In [1]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pandas to spark").getOrCreate()

In [14]:
df = pd.read_csv("Acoustic_Extinguisher_Fire_Dataset.csv")
df["FUEL"] = df["FUEL"].astype("category")
df["FUEL"] = df["FUEL"].cat.codes
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
df.head()

,SIZE,FUEL,DISTANCE,DESIBEL,AIRFLOW,FREQUENCY,STATUS
0,1,0,10,96,0.0,75,0
1,1,0,10,96,0.0,72,1
2,1,0,10,96,2.6,70,1
3,1,0,10,96,3.2,68,1
4,1,0,10,109,4.5,67,1


In [15]:
cl = df.columns[:-1]
cl = list(cl)

In [16]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df[cl] = ss.fit_transform(df[cl])

In [17]:
df1 = spark.createDataFrame(df)
df1.head()

Row(SIZE=-1.3774219384009612, FUEL=-1.1852755159795074, DISTANCE=-1.6431676725154982, DESIBEL=-0.0464415350316309, AIRFLOW=-1.4728852648142814, FREQUENCY=2.0722013081031, STATUS=0)

In [18]:
from pyspark.ml.feature import VectorAssembler
  
vec_assembler = VectorAssembler(inputCols = cl,
                                outputCol='features')

In [19]:
final_data = vec_assembler.transform(df1)
final_data.select(['features','STATUS']).show(5)

+--------------------+------+
|            features|STATUS|
+--------------------+------+
|[-1.3774219384009...|     0|
|[-1.3774219384009...|     1|
|[-1.3774219384009...|     1|
|[-1.3774219384009...|     1|
|[-1.3774219384009...|     1|
+--------------------+------+
only showing top 5 rows



In [20]:
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed = 42)

In [21]:
from pyspark.ml.classification import LinearSVC
svc = LinearSVC(labelCol='STATUS', maxIter=50)
model = svc.fit(train_data)

In [24]:
y_pred = model.transform(test_data)
y_pred.show()

+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+----------+
|               SIZE|               FUEL|           DISTANCE|             DESIBEL|             AIRFLOW|           FREQUENCY|STATUS|            features|       rawPrediction|prediction|
+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+----------+
|-1.3774219384009612|-1.1852755159795074|-1.6431676725154982|  -1.393842985846518|-0.20600225654311807| -1.3664323606569737|     1|[-1.3774219384009...|[-1.9693001981756...|       1.0|
|-1.3774219384009612|-1.1852755159795074|-1.6431676725154982| -0.7813877809306603|   1.039766034923526| -0.9366031520619645|     1|[-1.3774219384009...|[-3.4492905784426...|       1.0|
|-1.3774219384009612|-1.1852755159795074|-1.6431676725154982| -0.5364056989